In [ ]:
# Imports
import pickle
import numpy as np
import keras
from keras.utils import np_utils
from music21 import instrument, note, stream, chord,converter
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation
import glob
import tensorflow as tf

In [ ]:
with open('/content/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

In [ ]:
pitch = sorted(set(i for i in notes))
n_vocab = len(pitch)


In [ ]:
note_int = dict((i, n) for n, i in enumerate(pitch))
seq_l = 100
input,output = [],[]
for i in range(len(notes) - seq_l):
    temp = notes[i:i+seq_l]
    input.append([note_int[j] for j in temp])
    output.append(note_int[notes[i+seq_l]])

norm_input = np.reshape(input, (-1, seq_l, 1))
norm_output = np.reshape(output, (-1, 1))
norm_input = norm_input / float(n_vocab)
norm_output = norm_output/float(n_vocab)
norm_output2 = norm_output*float(n_vocab)
norm_output1 = keras.utils.np_utils.to_categorical(norm_output2)

In [ ]:
model = Sequential()
model.add(LSTM(512,input_shape=(norm_input.shape[1], norm_input.shape[2]),recurrent_dropout=0.3,
        return_sequences=True))
model.add(LSTM(300))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.25,patience=1,verbose=1,)
Adam = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(loss='categorical_crossentropy', optimizer=Adam)
model.fit(norm_input,norm_output1  ,epochs=10, batch_size=64, verbose = 1,callbacks = [callbacks])

Epoch 1/10
435/435 [==============================] - 165s 368ms/step - loss: 4.8674 - lr: 0.0500
Epoch 2/10
435/435 [==============================] - ETA: 0s - loss: 4.8982
Epoch 2: ReduceLROnPlateau reducing learning rate to 0.012500000186264515.
435/435 [==============================] - 160s 368ms/step - loss: 4.8982 - lr: 0.0500
Epoch 3/10
435/435 [==============================] - 160s 369ms/step - loss: 4.6651 - lr: 0.0125
Epoch 4/10
435/435 [==============================] - 161s 370ms/step - loss: 4.6536 - lr: 0.0125
Epoch 5/10
435/435 [==============================] - 162s 372ms/step - loss: 4.6469 - lr: 0.0125
Epoch 6/10
435/435 [==============================] - ETA: 0s - loss: 4.6483
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0031250000465661287.
435/435 [==============================] - 159s 366ms/step - loss: 4.6483 - lr: 0.0125
Epoch 7/10
435/435 [==============================] - 161s 369ms/step - loss: 4.6036 - lr: 0.0031
Epoch 8/10
435/435 [===========

In [ ]:
model.save('song2_model_512512512densedense_200e.h5')
# model = keras.models.load_model('hi.h5')

from google.colab import files
files.download('song2_model_512512512densedense_200e.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
start = np.random.randint(0, len(input)-1)
int_note = dict((n, i) for n, i in enumerate(pitch))
p = input[start]
pred = []
for i in range(500):
    temp = np.reshape(p, (1, len(p), 1))
    temp = temp / float(n_vocab)

    temp_p = model.predict(temp, verbose=0)

    index = np.argmax(temp_p)
    try:
      result = int_note[index]
    except:
      result = int_note[0]
    pred.append(result)

    p.append(index)
    p = p[1:len(p)]



In [ ]:
offset = 0
pred_notes = []
for i in pred:
        # chord
        if ('.' in i) or i.isdigit():
            chord_l = i.split('.')
            notes1 = []
            for j in chord_l:
                temp = note.Note(int(j))
                temp.storedInstrument = instrument.Piano()
                notes1.append(temp)
            new_chord = chord.Chord(notes1)
            new_chord.offset = offset
            pred_notes.append(new_chord)
        # note
        else:
            temp = note.Note(i)
            temp.offset = offset
            temp.storedInstrument = instrument.Piano()
            pred_notes.append(temp)

        offset += 0.5

midi_stream = stream.Stream(pred_notes)
midi_stream.write('midi', fp='test_output1.mid')


'test_output1.mid'

In [ ]:
files.download('test_output1.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
len(set(pred))

37

In [ ]:
model = Sequential()
model.add(LSTM(512,input_shape=(norm_input.shape[1], norm_input.shape[2]),recurrent_dropout=0.3,
        return_sequences=True))
model.add(LSTM(300))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.5,patience=2,verbose=1)
# Adam = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(norm_input,norm_output1  ,epochs=10, batch_size=64, verbose = 1,callbacks = [callbacks])

Epoch 1/10
435/435 [==============================] - 173s 387ms/step - loss: 4.7039 - lr: 0.0010
Epoch 2/10
435/435 [==============================] - 161s 369ms/step - loss: 4.4435 - lr: 0.0010
Epoch 3/10
435/435 [==============================] - 162s 371ms/step - loss: 4.4233 - lr: 0.0010
Epoch 4/10
435/435 [==============================] - 161s 370ms/step - loss: 4.4001 - lr: 0.0010
Epoch 5/10
435/435 [==============================] - 161s 370ms/step - loss: 4.4378 - lr: 0.0010
Epoch 6/10
435/435 [==============================] - ETA: 0s - loss: 4.4094
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
435/435 [==============================] - 160s 367ms/step - loss: 4.4094 - lr: 0.0010
Epoch 7/10
435/435 [==============================] - 158s 363ms/step - loss: 4.3868 - lr: 5.0000e-04
Epoch 8/10
435/435 [==============================] - 161s 368ms/step - loss: 4.3656 - lr: 5.0000e-04
Epoch 9/10
435/435 [==============================] - 163s 373ms/s

In [ ]:
callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.5,patience=2,verbose=1)
model.fit(norm_input,norm_output1  ,epochs=50, batch_size=64, verbose = 1,callbacks = [callbacks])

Epoch 1/50
435/435 [==============================] - 168s 375ms/step - loss: 2.3905 - lr: 5.0000e-04
Epoch 2/50
435/435 [==============================] - 161s 371ms/step - loss: 2.3435 - lr: 5.0000e-04
Epoch 3/50
435/435 [==============================] - 162s 372ms/step - loss: 2.2985 - lr: 5.0000e-04
Epoch 4/50
435/435 [==============================] - 164s 377ms/step - loss: 2.2511 - lr: 5.0000e-04
Epoch 5/50
435/435 [==============================] - 162s 372ms/step - loss: 2.2038 - lr: 5.0000e-04
Epoch 6/50
435/435 [==============================] - 161s 371ms/step - loss: 2.1711 - lr: 5.0000e-04
Epoch 7/50
435/435 [==============================] - 161s 370ms/step - loss: 2.1253 - lr: 5.0000e-04
Epoch 8/50
435/435 [==============================] - 162s 373ms/step - loss: 2.0948 - lr: 5.0000e-04
Epoch 9/50
435/435 [==============================] - 163s 374ms/step - loss: 2.0633 - lr: 5.0000e-04
Epoch 10/50
435/435 [==============================] - 162s 373ms/step - loss: 2.0

In [ ]:
#30
callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.5,patience=1,verbose=1)
model.fit(norm_input,norm_output1  ,epochs=10, batch_size=64, verbose = 1,callbacks = [callbacks])

Epoch 1/10
435/435 [==============================] - 343s 762ms/step - loss: 4.5930 - lr: 5.0000e-04
Epoch 2/10
435/435 [==============================] - ETA: 0s - loss: 4.5939
Epoch 2: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
435/435 [==============================] - 341s 783ms/step - loss: 4.5939 - lr: 5.0000e-04
Epoch 3/10
435/435 [==============================] - 331s 762ms/step - loss: 4.5914 - lr: 2.5000e-04
Epoch 4/10
435/435 [==============================] - 327s 752ms/step - loss: 4.5899 - lr: 2.5000e-04
Epoch 5/10
435/435 [==============================] - ETA: 0s - loss: 4.6139
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
435/435 [==============================] - 324s 744ms/step - loss: 4.6139 - lr: 2.5000e-04
Epoch 6/10
435/435 [==============================] - 325s 747ms/step - loss: 4.5063 - lr: 1.2500e-04
Epoch 7/10
435/435 [==============================] - 327s 751ms/step - loss: 4.4748 - lr: 1.2500e-04
Ep

In [ ]:
#40
callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.5,patience=1,verbose=1)
model.fit(norm_input,norm_output1  ,epochs=10, batch_size=64, verbose = 1,callbacks = [callbacks])

Epoch 1/10
435/435 [==============================] - 346s 767ms/step - loss: 4.4041 - lr: 1.2500e-04
Epoch 2/10
435/435 [==============================] - 344s 792ms/step - loss: 4.3781 - lr: 1.2500e-04
Epoch 3/10
435/435 [==============================] - 332s 764ms/step - loss: 4.3609 - lr: 1.2500e-04
Epoch 4/10
435/435 [==============================] - 334s 769ms/step - loss: 4.3312 - lr: 1.2500e-04
Epoch 5/10
435/435 [==============================] - 337s 775ms/step - loss: 4.3080 - lr: 1.2500e-04
Epoch 6/10
435/435 [==============================] - 337s 774ms/step - loss: 4.2850 - lr: 1.2500e-04
Epoch 7/10
435/435 [==============================] - 334s 768ms/step - loss: 4.2556 - lr: 1.2500e-04
Epoch 8/10
435/435 [==============================] - 336s 772ms/step - loss: 4.2228 - lr: 1.2500e-04
Epoch 9/10
435/435 [==============================] - 340s 782ms/step - loss: 4.1909 - lr: 1.2500e-04
Epoch 10/10
435/435 [==============================] - 332s 764ms/step - loss: 4.1

In [ ]:
#50
callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.5,patience=1,verbose=1)
model.fit(norm_input,norm_output1  ,epochs=10, batch_size=64, verbose = 1,callbacks = [callbacks])

Epoch 1/10
435/435 [==============================] - 331s 760ms/step - loss: 4.1144 - lr: 1.2500e-04
Epoch 2/10
435/435 [==============================] - 331s 761ms/step - loss: 4.0824 - lr: 1.2500e-04
Epoch 3/10
435/435 [==============================] - 331s 762ms/step - loss: 4.0508 - lr: 1.2500e-04
Epoch 4/10
435/435 [==============================] - 330s 760ms/step - loss: 4.0110 - lr: 1.2500e-04
Epoch 5/10
435/435 [==============================] - 333s 766ms/step - loss: 3.9874 - lr: 1.2500e-04
Epoch 6/10
435/435 [==============================] - 334s 767ms/step - loss: 3.9507 - lr: 1.2500e-04
Epoch 7/10
435/435 [==============================] - 332s 763ms/step - loss: 3.9165 - lr: 1.2500e-04
Epoch 8/10
435/435 [==============================] - 330s 760ms/step - loss: 3.8872 - lr: 1.2500e-04
Epoch 9/10
435/435 [==============================] - 331s 760ms/step - loss: 3.8520 - lr: 1.2500e-04
Epoch 10/10
435/435 [==============================] - 342s 785ms/step - loss: 3.8

In [ ]:
#112
callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.5,patience=1,verbose=1)
model.fit(norm_input,norm_output1  ,epochs=12, batch_size=64, verbose = 1,callbacks = [callbacks])

Epoch 1/12
435/435 [==============================] - 363s 807ms/step - loss: 1.8501 - lr: 1.2500e-04
Epoch 2/12
435/435 [==============================] - 344s 791ms/step - loss: 1.8348 - lr: 1.2500e-04
Epoch 3/12
435/435 [==============================] - 342s 787ms/step - loss: 1.8053 - lr: 1.2500e-04
Epoch 4/12
435/435 [==============================] - 349s 803ms/step - loss: 1.7695 - lr: 1.2500e-04
Epoch 5/12
435/435 [==============================] - 350s 804ms/step - loss: 1.7481 - lr: 1.2500e-04
Epoch 6/12
435/435 [==============================] - 346s 796ms/step - loss: 1.7273 - lr: 1.2500e-04
Epoch 7/12
435/435 [==============================] - 343s 789ms/step - loss: 1.6993 - lr: 1.2500e-04
Epoch 8/12
435/435 [==============================] - 343s 787ms/step - loss: 1.6607 - lr: 1.2500e-04
Epoch 9/12
435/435 [==============================] - 347s 799ms/step - loss: 1.6287 - lr: 1.2500e-04
Epoch 10/12
435/435 [==============================] - 346s 795ms/step - loss: 1.6

In [ ]:
#200
callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.5,patience=1,verbose=1)
model.fit(norm_input,norm_output1  ,epochs=25, batch_size=64, verbose = 1,callbacks = [callbacks])

Epoch 1/25
435/435 [==============================] - 331s 737ms/step - loss: 0.5206 - lr: 1.2500e-04
Epoch 2/25
435/435 [==============================] - 324s 744ms/step - loss: 0.5189 - lr: 1.2500e-04
Epoch 3/25
435/435 [==============================] - 324s 746ms/step - loss: 0.4947 - lr: 1.2500e-04
Epoch 4/25
435/435 [==============================] - ETA: 0s - loss: 0.4995
Epoch 4: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
435/435 [==============================] - 345s 793ms/step - loss: 0.4995 - lr: 1.2500e-04
Epoch 5/25
435/435 [==============================] - 325s 747ms/step - loss: 0.4241 - lr: 6.2500e-05
Epoch 6/25
435/435 [==============================] - 322s 740ms/step - loss: 0.4008 - lr: 6.2500e-05
Epoch 7/25
435/435 [==============================] - 331s 760ms/step - loss: 0.3858 - lr: 6.2500e-05
Epoch 8/25
435/435 [==============================] - 330s 758ms/step - loss: 0.3798 - lr: 6.2500e-05
Epoch 9/25
435/435 [=======================